In [14]:
import pandas as pd
from sklearn.neighbors import BallTree

# Load Data
bus_stops = pd.read_excel('/Users/tyleratlas/Desktop/BC Transit Project/BUS_STOPS.xlsx')
data = pd.read_csv('/Users/tyleratlas/Desktop/BC Transit Project/JOINED_DATA.csv')

# Rename Lat/Long columns
bus_stops.rename(columns={'Latitude': 'stop_latitude', 'Longitude': 'stop_longitude'}, inplace=True)
data.rename(columns={'Unnamed: 0': 'id', 'Latitude': 'bus_latitude', 'Longitude': 'bus_longitude'}, inplace=True)

# Create arrays for coordinates
stop_coords = bus_stops[['stop_latitude', 'stop_longitude']].values
bus_coords = data[['bus_latitude', 'bus_longitude']].values

# Build a BallTree for efficient nearest neighbor search
tree = BallTree(bus_stop_coords, leaf_size=15, metric='haversine')

# Query the nearest neighbors
distances, indices = tree.query(transaction_coords, k=1)

# Create results DataFrame
nearest_bus_stops = pd.DataFrame({
    'id': data['id'].values,
    'transaction_latitude': data['bus_latitude'].values,
    'transaction_longitude': data['bus_longitude'].values,
    'bus_stop_latitude': bus_stops['stop_latitude'].iloc[indices.flatten()].values,
    'bus_stop_longitude': bus_stops['stop_longitude'].iloc[indices.flatten()].values,
    'Stop Name': bus_stops['Stop Name'].iloc[indices.flatten()].values,
    'distance': distances.flatten()
})

# Exclude specific bus stops
exclude_bus_stops = ['20 Hawley St', 'Chenago/State', 'Chenango/Lyon', 'Tremont/State St on ramp']
nearest_bus_stops = nearest_bus_stops[~nearest_bus_stops['Stop Name'].isin(exclude_bus_stops)]

# Merge with joined data
nearest_bus_stops = pd.merge(nearest_bus_stops, data[['id', 'Address', 'date', 'time.x', 'am_pm',
                                                      'bus_num', 'Route', 'movement']], on='id', how='left')

# Save to CSV
nearest_bus_stops.to_csv('/Users/tyleratlas/Desktop/BC Transit Project/NEAREST_BUS_STOP.csv', index=False)

/var/folders/5d/frvqls_x3yl54h_319g61bm80000gn/T/ipykernel_7124/1497314734.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/tyleratlas/Desktop/BC Transit Project/JOINED_DATA.csv')
